In [1]:
import pandas as pd
import numpy as np
import os
import timeit
 
start_time = timeit.default_timer() # 시작 시간 체크
 

min_dataV = 0
max_dataV = [0,0,0]

#19년 20년 월별 데이터 생성
data2019 = {}
data2020 = {}

def csv_to_dataFrame(year,dataF):
    global min_dataV,max_dataV
    year = str(year)
    
    for indx,csvF in enumerate(os.listdir('./seoul_subway_data/{}'.format(year))):
        #19년도 데이터 안에 이미 데이터가 있으면 넘어가고 없으면 기본값으로 추가
        if not 'data' + csvF[-10:-4] in dataF.keys():
            dataF.setdefault('data' + csvF[-10:-4])
        else:
            continue

        try:
            data = pd.read_csv('./seoul_subway_data/' + year + '/' + csvF,encoding = 'UTF-8',index_col=False)
        except Exception as e:
            data = pd.read_csv('./seoul_subway_data/' + year + '/' + csvF,encoding = 'EUC-KR',index_col=False)
        dataF['data' + csvF[-10:-4]] = data
        
    #데이터에서 불필요한 데이터 조정
    for dataYM in dataF.keys():
        if '역ID' in dataF[dataYM].columns:
            dataF[dataYM].drop('등록일자',axis=1,inplace = True)
            dataF[dataYM].rename(columns = {'하차총승객수' : '등록일자'},inplace = True)
            dataF[dataYM].rename(columns = {'승차총승객수' : '하차총승객수'},inplace = True)
            dataF[dataYM].rename(columns = {'역명' : '승차총승객수'},inplace = True)
            dataF[dataYM].rename(columns = {'역ID' : '역명'},inplace = True)
            
        dataF[dataYM]['유동인구수'] = dataF[dataYM]['승차총승객수'] + dataF[dataYM]['하차총승객수']

#지도를 어떻게 표현할지에 대해 필요있을 수 있는 코드
#(모든 월별 지도의 최대 레전드 값을 고정을 시켜놓으면 전체적으로 볼때는 알아보기 쉬우나 특정 몇몇 지역들이 지나치게 흐려짐)
#(월별로 지도의 최대 레전드 값을 다르게 하면 색은 눈에 띄게 표현 가능하나 전체적인 지도를 볼 때는 계속해서 레전드의 최댓값을 확인할 필요가 잇음)
#         if max_dataV[0] < dataF[dataY]['승차총승객수'].max():
#             max_dataV[0] = dataF[dataY]['승차총승객수'].max()
        
#         if max_dataV[1] < dataF[dataY]['하차총승객수'].max():
#             max_dataV[1] = dataF[dataY]['하차총승객수'].max()
            
#         if max_dataV[2] < dataF[dataY]['유동인구수'].max():
#             max_dataV[2] = dataF[dataY]['유동인구수'].max()           
            
        #후우....... 데이터 최적화
        #중복되는 역명(환승선들) 합쳐서 그 역의 호선 갯수만큼 나누기
        #사용일자/승차/하차/유동인구수 까지만 남기고 
        a = dataF[dataYM].groupby('사용일자')
        t_df = pd.DataFrame()
        tmp = {}
        for i,t in a:
            try:
                t = t.reset_index()
                b_c = t.groupby('역명').count()[['승차총승객수','하차총승객수','유동인구수']]
                b = t.groupby('역명').sum()[['승차총승객수','하차총승객수','유동인구수']]
                k = b.divide(b_c).rename_axis('역명').reset_index()        
                k['사용일자'] = t['사용일자']
                tmp[i] = k
                tmp[i] = pd.DataFrame(tmp[i])
                
                du_tmp = tmp[i]['역명']

                #중복확인(혹시라도 역명이 동일한게 있는지 확인) / 사실 불필요한 코드
#                 for j,x in enumerate(du_tmp.duplicated()):
#                     try:
#                         if x == True:
#                             raise Exception(du_tmp[j])
#                     except Exception as e:
#                         print(e)
#                         raise Exception()
#                         break           
            except Exception as e: 
                break
            t_df = pd.concat([t_df,tmp[i]],axis = 0,ignore_index = True)

        dataF[dataYM] = t_df 

csv_to_dataFrame(2019,data2019)
csv_to_dataFrame(2020,data2020)

terminate_time = timeit.default_timer() # 종료 시간 체크  
 
print("%f초 걸렸습니다." % (terminate_time - start_time)) 



<h1>크롤링으로 구 csv 만들기</h1>

In [ ]:
# from selenium import webdriver
# import time
# import os, csv

# option = webdriver.ChromeOptions()
# option.add_argument('headless')

# # os.path.abspath('chromedriver')

# driver = webdriver.Chrome(os.path.abspath('chromedriver'),options=option)
# driver.get('https://gits.gg.go.kr/gtdb/web/trafficDb/railRoad/TransitSWPass.do')

# time.sleep(3)

# def setRadio():
#     driver.find_element_by_xpath('//*[@id="radio1"]').click()

# def setSelect1(sel1):
#     driver.find_element_by_xpath("//select[@name='select1']/option[text()='%s']" %sel1).click()
    
# def setSelect2(sel2):
#     driver.find_element_by_xpath("//select[@name='select2']/option[text()='%s']" %sel2).click()

# #soup로 고정된 html 값에서 radio1 버튼 찾아와서 클릭하기
# setRadio()

# #rad 버튼 클릭후 동적으로 변경된 웹페이지에서 select1아이디 값 가져오기
# select1 = driver.find_element_by_id('select1')
# t_sel1 = select1.text.split('\n')
# sel1 = []
# for i,t in enumerate(t_sel1):
#     if not t.strip() == '':
#         sel1.append(t.strip())

# #시 - 역명 넣을 csv파일 생상(기존에 있는건 삭제)
# if os.path.exists('./seoul_subway_data/구_역명.csv'):
#     os.remove('./seoul_subway_data/구_역명.csv')

#encoding 설정 안해주면 mac은 되는데 window는 안될 때 있음 (조심)
# with open('./seoul_subway_data/구_역명.csv','w',encoding = 'utf-8') as csvFile:
#     writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])    
#     writer.writeheader()
    
# #sel1 값을 넣은 후 sel2값 넣고 검색 후 list값들을 가져온 다음 데이터 정리
# for i in sel1:
#     setSelect1(i)
    
#     sel2 = driver.find_element_by_id('select2')
#     t_sel2 = sel2.text.split('\n')
#     for a in range(len(t_sel2)):
#         t_sel2[a] = t_sel2[a].strip()
#         #전체 선택 혹은 비어있는 경우는 넘어가기 (행정 구역만 필요하니까)
#         if '전체선택' == t_sel2[a] or '' == t_sel2[a]:
#             continue
        
#         setSelect2(t_sel2[a])
#         driver.find_element_by_xpath('//*[@id = "search"]').click()
#         li_loc = (driver.find_element_by_id('selList')).text.split('\n')

#         for i in li_loc:
#             #가져온 list값은 매봉(1호선) 이런식으로 돼있으므로 ()를 기준으로 역이름과 호선명 나눠주기
#             st_idx = i.find('(') + 1
#             f_idx = i.find(')')
#             t_sel2[a] = t_sel2[a].replace('_','')
#             #csv파일에 바로 넣기
#             with open('./seoul_subway_data/구_역명.csv','a',encoding = 'utf-8') as csvFile:
#                 writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])
#                 writer.writerow({'구':t_sel2[a],'역명':i[:st_idx-1]})
                
# driver.quit()

<h1>데이터 정리 최종</h1>

In [2]:
gu_sta_name = pd.read_csv('./seoul_subway_data/구_역명.csv',encoding='utf-8')

def insert_gu(dataF):
    global min_dataV,max_dataV
    newData_min = {'사용일자':'', '역명':'', '승차총승객수':min_dataV, '하차총승객수':min_dataV, '유동인구수':min_dataV}
    newData_max = {'사용일자':'', '역명':'', '승차총승객수':max_dataV[0], '하차총승객수':max_dataV[1], '유동인구수':max_dataV[2]}
    #데이터에 구 추가
    for dataY in dataF.keys():
        if not '구' in dataF[dataY].columns:    
            dataF[dataY] = pd.merge(dataF[dataY],gu_sta_name,on='역명',how = 'inner')

            if True in dataF[dataY].duplicated():
                dataF[dataY] = dataF[dataY].drop_duplicates()
                dataF[dataY].reset_index(inplace=True,drop = True)

        dataF[dataY] = dataF[dataY].sort_values(['사용일자','역명'],ascending = True).reset_index(drop = True)             
        dataF[dataY] = dataF[dataY].append(newData_min,ignore_index=True)
#         dataF[dataY] = dataF[dataY].append(newData_max,ignore_index=True)
    
insert_gu(data2019)
insert_gu(data2020)

DATA = {}
if not 'data2019' in DATA.keys():
    DATA['data2019'] = data2019
if not 'data2020' in DATA.keys():
    DATA['data2020'] = data2020

<h1>그냥 바로 해버리면 전국에 칠해지니까 서울지역으로 json 범위 축소 </h1><br><p>근데 왜 한번에 안되고 여러번에 거쳐야 되는지는 모르겠음...</p>

In [3]:
import json
import folium
import webbrowser
import time
import matplotlib.pyplot as plt

geo_path = './map/skorea-municipalities-2018-geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

#json 데이터 안다듬어주면 전국단위로 난리남
print('json 처리 전 지역 수 : ',len(geo_str['features']))
min_len_geoStr = False
k = [-1,-2]
#서울 지역 코드 값 = 1로 시작
#왜인지는 모르겠는데 한번에 처리가 안되서 이전 카운트 개수하고 현재 카운트 개수에 차이가 없을 때 break
while min_len_geoStr == False:
    if k[0] == k[1]:
        min_len_geoStr = True
    k[0] = len(geo_str['features'])
    for i,t in enumerate(geo_str['features']):
        if not t['properties']['code'][0] == '1':
            del geo_str['features'][i]
    k[1] = len(geo_str['features'])        

print('json 처리 후 지역 수 : ',len(geo_str['features']))



json 처리 전 지역 수 :  250
json 처리 후 지역 수 :  25


<h1>지하철 클래스 생성</h1>

In [ ]:
class subwayData:
    global geo_str,DATA
    
    """
     --------------------
    |    Map Function    |
     --------------------
    """ 

    def __init__(self,dataF,ym):      
        #map 설정
        self.map=folium.Map(location=[37.5502, 126.982], zoom_start=11, min_zoom=11,
                    tiles='stamentoner')
        
        #font 설정
        plt.rcParams['font.family'] = 'Apple Gothic'        
        
        self.dataF = dataF
        self.ym = ym
        self.dataYM = 'data' + ym
        
        try:
            self.dataF[self.dataYM]
        except:
            print('No data')
            return False
        
        self.DrawGuMap()

    #지하철로 구 유동인구 파악
    def DrawGuMap(self):
        #맵 그려주기
        folium.Choropleth(
            geo_data=geo_str,
            data=self.dataF[self.dataYM], 
            columns=['구','유동인구수'], 
            key_on='feature.properties.name',
            fill_color='PuRd',
            legend_name='구별 지하철 유동인구 수'
        ).add_to(self.map)
        self.SaveMap()
    
    #지하철 역별 유동인구 파악
    def DrawSubMap(self):
        #맵 그려주기
        folium.Choropleth(
            geo_data=geo_str,
            data=self.dataF[self.dataYM], 
            columns=['역명','유동인구수'], 
            key_on='feature.properties.name',
            fill_color='PuRd',
            legend_name='구별 지하철 유동인구 수'
        ).add_to(self.map)
        self.SaveMap()
    
    #그린 맵 파일 확인 후 저장 (나중에 html파일을 웹에 올려야 되서 따로 저장이 필요하긴함)
    def SaveMap(self):
        #쥬피터에서 보여주면 한글 깨져서 동일 경로 map 디렉토리에 해당 맵을 만들어줌
        if os.path.exists('./map'):
            if os.path.exists('./map/{}.html'.format(self.ym)):
                os.remove('./map/{}.html'.format(self.ym))
        else:
            os.makedirs('./map')
        self.map.save('./map/{}.html'.format(self.ym))    
        
    #파일 삭제
    def DelMap(self):
        if os.path.exists('./map/{}.html'.format(self.ym)):
            os.remove('./map/{}.html'.format(self.ym))        

    #파일 열기 
    def OpenMap(self):
        try:
            if os.path.exists('./map/{}.html'.format(self.ym)):
                BASE_DIR = os.path.dirname(os.path.abspath('map/{}.html'.format(self.ym)))
                webbrowser.open('file:' + os.path.join(BASE_DIR,'{}.html'.format(self.ym)))
                #파일 삭제전에 잠시 텀을 줘서 저장한게 보이게끔 함
                #나중에 파일 삭제 관련해서 다시 다뤄야할 때를 대비 (이 코드 안에서 삭제를 없애야 될 수도 있음 / 그래야 html에서 가져올수 있으니까)
                time.sleep(5)
                self.DelMap()
            else:
                raise Exception(self.ym + ' file is not exist')

        except Exception as e:
            print(e)
            return False
        
    """
     ----------------------
    |    Graph Function    |
     ----------------------
    """        
        
        
    #graph를 하나로 묶고 subplots을 쓸지 나눠서 각각의 fig를 만들고 html에서 div를 통해서 구분을 지어줄지 결정해야 됨
    def DrawBarGraph(self):
        #임시로 사이즈는 대충 둠
        print(self.dataF[self.dataYM])
        plt.figure(figsize = (20,20))
        plt.bar(self.dataF[self.dataYM]['역명'],self.dataF[self.dataYM])
        plt.xticks(rotation = 90)
        plt.titles('{} 유동인구 데이터'.format(dataYM))
        plt.show()
    
    def DrawLineGraph(self):
        pass

<h1>클라이언트 딴에서 실행</h1>

In [7]:
def setSubwayData(dataF,ym):
    mySub_data = subwayData(dataF,ym)
    return mySub_data

#DATA = server와 연동해서 server의 DATA값을 가져와야 함
if __name__ == '__main__':
    global DATA
    while True:
        dataYM,selT = map(str,input('Input dataYear (ex:202010) & Select Type (1:승차 / 2:하차 / 3:유동인구)\n').split(' '))
        try:
            #DATA에 들어있는 data2019 data2020의 연도와 인풋 받은 dataYM를 비교해서 입력 형식이 틀리면 출력
            for datayear in DATA.keys():
                if not len(dataYM) == 6 or not dataYM[:4] in datayear:
                    continue
                myMap = setMap(DATA[datayear],dataYM,selT)
                
            #map이 열리면 break
            if not myMap.openMap() == False:
                break
                
            #안열리면 입력 형식에 문제있음
            print("{} isn't correct year".format(dataYM))                
        except:
            break





Input dataYear (ex:202010) & Select Type (1:승차 / 2:하차 / 3:유동인구)
202005 1


In [9]:
DATA['data2020']['data202005']

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,유동인구수,구
0,20200501,중앙선,운길산,1349.0,1358.0,20200504,9.0,남양주시
1,20200502,중앙선,운길산,1467.0,1554.0,20200505,87.0,남양주시
2,20200503,중앙선,운길산,1369.0,1345.0,20200506,-24.0,남양주시
3,20200504,중앙선,운길산,928.0,885.0,20200507,-43.0,남양주시
4,20200505,중앙선,운길산,1338.0,1241.0,20200508,-97.0,남양주시
...,...,...,...,...,...,...,...,...
14189,20200529,중앙선,양수,1850.0,1930.0,20200601,80.0,양평군
14190,20200530,중앙선,양수,2234.0,2374.0,20200602,140.0,양평군
14191,20200531,중앙선,양수,1911.0,1577.0,20200603,-334.0,양평군
14192,,,,0.0,0.0,,0.0,NaN
